In [1]:
import os
import json
import csv
from dotenv import load_dotenv

from s2ag_corpus.sql import CREATE_PAPERS_TABLE_WITH_KEYS

In [2]:
load_dotenv()
base_dir = os.getenv("BASE_DIR")

In [3]:
from s2ag_corpus.database_catalogue import local_connection

connection = local_connection()

def drop_and_replace_papers_table():
    with connection.cursor() as cursor:
        cursor.execute('drop table if exists papers')
        cursor.execute(CREATE_PAPERS_TABLE_WITH_KEYS)

In [4]:
drop_and_replace_papers_table()
    

In [7]:
test_file = base_dir+'/2024-04-02/papers/first10000papers'
transfer_file = base_dir+'/2024-04-02/papers/transfer.csv'
with open(test_file) as f:
    jason_dictionaries = [(line, json.loads(line)) for line in f.readlines()]
    records = [(jd['corpusid'], line.strip()) for  line, jd in jason_dictionaries]
    with open(transfer_file,'w') as csvf:
        writer = csv.writer(csvf, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
        for record in records:
            writer.writerow(record)

In [8]:
with open(transfer_file,'r') as csvf:
    with connection.cursor() as cursor:
        cursor.copy_from(csvf, 'papers', sep=',', null='')
    connection.commit()